In [171]:
from matplotlib.pyplot import imshow, imsave

import numpy as np
import cv2

from skimage.transform import resize
from skimage.io import imread
from skimage.filters.rank import mean
from skimage.exposure import cumulative_distribution, histogram

%matplotlib inline

In [172]:
HEIGHT, WIDTH = 66, 200

image = imread('res.jpg')
image = resize(image, (HEIGHT, WIDTH))

mean = image.mean()
print('mean: {0}'.format(mean))

for i in range(3):
    mean = image[:, :, i].mean()
    print('mean {0}: {1}'.format(i, mean))

mean: 0.5400803129332541
mean 0: 0.5360609031491383
mean 1: 0.5531256684491976
mean 2: 0.5310543672014258


/home/hippolyte/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hippolyte/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [173]:
def auto_brightness(image):
    # compute mean for all channels to determine if the image
    # is bright or dark
    mean = image.mean()

    # the value is arbitrary
    bright = mean > 0.4
    ideals =  [0.035, 0.11, 0.011]

    # compute mean for each channel
    for i, ideal in enumerate(ideals):
        if bright:
            ideal = np.exp(ideal - 1)
        mean = image[:, :, i].mean()
        image[:, :, i] = image[:, :, i] * (ideal / mean)
        print('mean {0}: {1}'.format(i, mean))


    # normalization of the image
    image[image > 1] = 1
    image = np.exp(image - 1)
    print('after: {0}, {1}, {2}'.format(image[:, :, 0].mean(),
                                        image[:, :, 1].mean(),
                                        image[:, :, 2].mean()))

    return image

In [174]:
def auto_contrast(image, lower_percentile=1, upper_percentile=98):
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    brightness = image[:,:,2]
    lower = np.percentile(brightness, lower_percentile)
    upper = np.percentile(brightness, upper_percentile)
    
    brightness[brightness <= lower] = 0
    brightness[brightness >= upper] = 255
    
    result = np.linalg.solve([[lower, 1], [upper, 1]], [0, 255])
    image[:, :, 2] = image[:, :, 2] * result[0] + result[1]
    return cv2.cvtColor(image, cv2.COLOR_HSV2RGB)

In [175]:
image = imread('dark.jpg')
image = auto_brightness(image)
imshow(image)

/home/hippolyte/.local/lib/python3.5/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


mean 0: 5.355085714285714
mean 1: 14.37737142857143
mean 2: 1.5176571428571428
after: inf, inf, inf


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f14e26ef488> (for post_execute):


ValueError: 3-dimensional arrays must be of dtype unsigned byte, unsigned short, float32 or float64

ValueError: 3-dimensional arrays must be of dtype unsigned byte, unsigned short, float32 or float64

<Figure size 432x288 with 1 Axes>

Test with real images

In [176]:
from os import listdir
from os.path import join

HEIGHT, WIDTH = 66, 200
READ_PATH = 'records_rev'
WRITE_PATH = 'results'

for file in listdir(READ_PATH):
    image = imread(join(READ_PATH, file))
    image = image[60:-20, :, :]
    image = resize(image, (HEIGHT, WIDTH))

    image = auto_brightness(image)
    
    imsave(join(WRITE_PATH, file), image)

mean 0: 0.019011809269162207
mean 1: 0.047761454788526975
mean 2: 0.010615184825077872
after: 0.38268190001469204, 0.4151542058848823, 0.37215304163466567
mean 0: 0.014583327706656586
mean 1: 0.042800808891049535
mean 2: 0.009528862601055113
after: 0.3835973563951595, 0.4154593588630018, 0.37218529425703156
mean 0: 0.00704552094024019
mean 1: 0.021119279920416286
mean 2: 0.003981107870145303
after: 0.3827449768513033, 0.41669758414702396, 0.3720739599567413
mean 0: 0.015582937215290159
mean 1: 0.04243564207133726
mean 2: 0.008189381560705089
after: 0.3829559669674049, 0.4141776212586254, 0.37219858439727543
mean 0: 0.012572785565098409
mean 1: 0.04130999612884641
mean 2: 0.007295948567673172
after: 0.3831849164244344, 0.413297294849905, 0.3721715488923613
mean 0: 0.0631310059147626
mean 1: 0.11121493792650211
mean 2: 0.012388166423594226
after: 0.38111590513131693, 0.41124437585395074, 0.37209558446585755
mean 0: 0.010686630115774504
mean 1: 0.04065268437494374
mean 2: 0.00642096319703

/home/hippolyte/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hippolyte/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
